In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/officetel/items/{item_id}"

referer = "https://zigbang.com/home/officetel/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/officetel/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [4]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'original_user_phone',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_email',
        'user_intro',
        'user_mobile',
        'user_name',
        'user_phone',
        'url'
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=7)
    

    for item in items:
        item = item['item']

        item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
        if maximum_updated < datetime.datetime.now() - item_updated:
            continue
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/officetel/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [7]:
items = {}

In [8]:
geos_seoul = [
    'wydm'
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [9]:
from itertools import islice

len(items)

sliceItem = dict(islice(items.items(), 3000))

len(sliceItem)

3000

In [10]:
print(sliceItem)

{'29003347': {}, '29229665': {}, '29164714': {}, '29011281': {}, '29167913': {}, '29211315': {}, '29120499': {}, '29197367': {}, '29149126': {}, '27719724': {}, '28925397': {}, '29193042': {}, '29231003': {}, '29136441': {}, '29175869': {}, '29083952': {}, '29208367': {}, '29082799': {}, '29178461': {}, '29199552': {}, '29236678': {}, '29118001': {}, '29174552': {}, '28922561': {}, '29136285': {}, '29058252': {}, '29163039': {}, '29205730': {}, '29103526': {}, '28957442': {}, '29166458': {}, '29183691': {}, '29188604': {}, '29146350': {}, '28789222': {}, '29076209': {}, '29207077': {}, '29162862': {}, '29116101': {}, '28947417': {}, '29089861': {}, '29224469': {}, '29117808': {}, '29084324': {}, '29103523': {}, '29202472': {}, '29052247': {}, '28941869': {}, '29213745': {}, '28965312': {}, '29209631': {}, '28938796': {}, '29133953': {}, '29157827': {}, '29205204': {}, '29230354': {}, '29225479': {}, '29189718': {}, '29212410': {}, '29232827': {}, '28196085': {}, '29128080': {}, '291835

In [11]:
result = describe_room_list(sliceItem)

In [12]:
len(result)

2900

In [13]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '13층',
  'address1': '서울시 은평구 대조동',
  'address2': '179-16',
  'address3': None,
  'agent_address1': '서울특별시 은평구 가좌로 334 상가동 7호(신사동, 효성해링턴플레이스)',
  'agent_comment': '',
  'agent_email': 'gojungo@naver.com',
  'agent_lat': 37.5981158955891,
  'agent_lng': 126.908770078234,
  'agent_local1': '서울시',
  'agent_local2': '은평구',
  'agent_mobile': '010-9124-5577',
  'agent_name': '효성탑공인중개사사무소(김희준)',
  'agent_no': 0,
  'agent_phone': '02-338-0700',
  'bjd_code': '',
  'bonbun_code': '179',
  'bubun_code': '16',
  'building': None,
  'building_type': '-',
  'contract': '서울특별시',
  'deposit': 24500,
  'description': '📀 매물 참고 TIP 📀\r\n'
                 '- 1.5룸 전세 매물 첫입주!\r\n'
                 '- 모든 시설물 최상 컨디션 ✨\r\n'
                 '- 주변 생활권 확보 잘 되어있어요! 👍\r\n'
                 '- 타워 주차형으로 주차걱정 끝🚗\r\n'
                 '- 모든 방 큰 창문으로 채광 및 환기 걱정 No ❗\r\n'
                 '- 전세 대출 문제 없습니다 👌\r\n'
                 '- 뷰 맛집!\n'
                 '21.10.07에 직방에 등록된 매물입니다.',
  'description_og': '

In [14]:
filtered_result = export_items(result)

In [15]:
len(filtered_result)


1826

In [16]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [17]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:9]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:10]
    
    return lng

In [18]:
geo_based_dict = {}

In [19]:
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)
    lat = get_location_lat(item)
    item['lat'] = lat
    lng = get_location_lng(item)
    item['lng'] = lng

In [21]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ 'address': '서울시 광진구 자양동 842-7 ',
  'address1': '서울시 광진구 자양동',
  'address2': '842-7',
  'address3': None,
  'agent_address1': '서울 동대문구 장안동 365-9',
  'agent_email': 'nhj1222@daum.net',
  'agent_lat': 37.56929058354021,
  'agent_lng': 127.070173783723,
  'agent_mobile': '010-6865-2299',
  'agent_name': '케이부동산공인중개사사무소(남혜정)',
  'agent_phone': '02-3394-5545',
  'building_type': '-',
  'deposit': 30000,
  'description': '건대입구역 인접\r\n'
                 '\r\n'
                 '신축 첫입주\r\n'
                 '\r\n'
                 '통창\r\n'
                 '\r\n'
                 '주변 인접 상권 편리(대형마트,편의시설)\r\n'
                 '\r\n'
                 '전세자금대출\r\n'
                 '\n'
                 '21.10.28에 직방에 등록된 매물입니다.',
  'elevator': '있음',
  'floor': '3층',
  'floor_all': '6층',
  'images': [ { 'count': 1,
                'index': 0,
                'url': 'https://ic.zigbang.com/ic/items/29229665/1.jpg'},
              { 'count': 2,
                'index': 1,
                'url': 'htt

In [22]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [23]:
with open('officetel.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'original_user_phone',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_email',
        'user_intro',
        'user_mobile',
        'user_name',
        'user_phone',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [ ]:
import pymysql
import csv

conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8')
curs = conn.cursor()
sql = "insert into officetel (id,deposit,agent_name,agent_mobile,floor,address,size,size_m2,status,room_type,manage_cost_inc,url,upddt) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, now())"
f = open('officetel.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

#title, description은 이모티콘 인식못하는 에러 -> 일단 제외
for line in rd:
    curs.execute(sql, (line[0], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13]))

conn.commit()
conn.close()
f.close()